In [2]:
#!/usr/bin/env python
#Config
from pyspark import SparkConf, SparkContext, HiveContext
#import re
#import numpy as np
#import pandas as pd
#import datetime
#from pyspark.mllib.regression import LabeledPoint
#from pyspark.mllib.feature import HashingTF
#from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
#import scipy.sparse as sps
#from pyspark.mllib.linalg import Vectors
#import sklearn


import sys
import itertools
from pyspark.mllib.recommendation import ALS
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

try:
    sc.stop()
except NameError:
    pass

conf = SparkConf().set("spark.executor.instances", 32).set("spark.driver.maxResultSize", "16g")
sc = SparkContext()
hc = HiveContext(sc)

In [3]:


# Load and parse the data
data = sc.textFile("/home/k.p.osminin/external_hdfs/ha_proc_20160915.txt")
ratings = hc.sql('''select id_num, urlfr_num, cnt/(count(*) over (partition by id_num)) as val 
                    from user_kposminin.visits_enum_20160412''') \
    .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))

# Build the recommendation model using Alternating Least Squares
rank = 200
numIterations = 10
model = ALS.train(ratings, rank, numIterations)

# Evaluate the model on training data
testdata = ratings.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))

# Save and load model
model.save(sc, "target/tmp/myCollaborativeFilter")
sameModel = MatrixFactorizationModel.load(sc, "target/tmp/myCollaborativeFilter")

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 4 times, most recent failure: Lost task 0.3 in stage 1.0 (TID 52, ds-hadoop-wk27p.tcsbank.ru): java.io.IOException: Cannot run program "/home/k.p.osminin/anaconda2/bin/python": error=2, No such file or directory
	at java.lang.ProcessBuilder.start(ProcessBuilder.java:1048)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:161)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:87)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:63)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:134)
	at org.apache.spark.api.python.PythonRunner.compute(PythonRDD.scala:101)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:70)
	at org.apache.spark.rdd.RDD.computeO